In [1]:
import os
import librosa
import pandas as pd
import numpy as np

In [2]:
wav_folder_path = "../../KEMDy20_v1_1/wav/"

In [3]:
wav_file_paths = []
for root, dirs, files in os.walk(wav_folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        if file_path.endswith(".wav"): # .wav 확장자만 저장
            wav_file_paths.append(file_path)

In [4]:
def get_feature(file_path: str, mfcc_len: int = 30, mean_signal_length: int = 88000):
    #file_path: Speech signal folder
    #mfcc_len: MFCC coefficient length
    #mean_signal_length: MFCC feature average length
        
    signal, fs = librosa.load(file_path)
    s_len = len(signal)

    if s_len < mean_signal_length:
        pad_len = mean_signal_length - s_len
        pad_rem = pad_len % 2
        pad_len //= 2
        signal = np.pad(signal, (pad_len, pad_len + pad_rem), 'constant', constant_values = 0)
    else:
        pad_len = s_len - mean_signal_length
        pad_len //= 2
        signal = signal[pad_len:pad_len + mean_signal_length]
    mfcc = librosa.feature.mfcc(y=signal, sr=fs, n_mfcc=mfcc_len)
    mfcc = mfcc.T
    feature = mfcc
    return feature

In [5]:
annotation_prob = pd.read_csv("../../KEMDy20_v1_1/labels/Annotation_prob.csv")
annotation_multi = pd.read_csv("../../KEMDy20_v1_1/labels/Annotation_multi.csv")

In [7]:
annotation_prob['Sess'] = annotation_prob['Segment ID'].str.extract('(Sess\d+)', expand=False)
annotation_multi['Sess'] = annotation_multi['Segment ID'].str.extract('(Sess\d+)', expand=False)

In [ ]:
folder_name = "../../KEMDy20_v1_1/mfcc_npz/"
if not os.path.exists(folder_name): 
    os.makedirs(folder_name)

for sess in np.unique(annotation_prob["Sess"]): 
    feature_list = []
    prob_label_list = []
    multi_label_list = []
    
    for wav_file_path in wav_file_paths:
        seg_id = wav_file_path.split("\\")[1][:-4]
        sess_id = seg_id.split("_")[0]
        
        if sess_id==sess:
            feature = get_feature(wav_file_path)
            feature_list.append(feature)
            
            prob_label = annotation_prob[annotation_prob["Segment ID"]==seg_id].values[0][1:-1].astype(float)
            prob_label_list.append(prob_label)
            
            multi_label = annotation_multi[annotation_multi["Segment ID"]==seg_id].values[0][1:-1].astype(float)
            multi_label_list.append(multi_label)
            
    feature_array = np.stack(feature_list)
    prob_label_array = np.stack(prob_label_list)
    multi_label_array = np.stack(multi_label_list)
    
    np.savez(folder_name+sess,
             features = feature_array,
             prob_labels = prob_label_array,
             multi_labels = multi_label_array)
    
    print(sess+" Complete !")

Sess01 Complete !
Sess02 Complete !
Sess03 Complete !
Sess04 Complete !
Sess05 Complete !
Sess06 Complete !
Sess07 Complete !
Sess08 Complete !
Sess09 Complete !
Sess10 Complete !
Sess11 Complete !
Sess12 Complete !
Sess13 Complete !
Sess14 Complete !
Sess15 Complete !
Sess16 Complete !
Sess17 Complete !
Sess18 Complete !
Sess19 Complete !
Sess20 Complete !
Sess21 Complete !
